In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [2]:
class OneToOneITU():
    def __init__(self, n, m, matrices , lbs=(0, 0), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 
       
        self.A_ij = matrices[0]
        self.B_ij = matrices[1]




        self.conditional_means_obs = None
        self.conditional_covs_obs = None

    def D_ij(self, U_i, V_j):
        return U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

Generate a a parametrized example

In [3]:
np.random.seed(10)
n_i = 300
m_j = 340

θ_true =  np.array([.5,.5,5,5])
k = len(θ_true)
X_ijk = np.random.normal(1,1, size = [n_i,m_j, k])
 

A = np.exp( X_ijk[:,:,:2] @ θ_true[:2])
B = X_ijk[:,:,2:4] @ θ_true[2:4]

In [4]:
B.mean()

9.985293898874836

In [5]:
def ITU_auction(self, tol):

    V_j = np.ones(self.m) * self.lb_V
    mu_ij = np.zeros([self.n,self.m+1], dtype= bool)

    unassigned = mu_ij.sum(1) == 0
    n_unassigned = unassigned.sum()
    iter = 0

    while n_unassigned > 0:
        
        i = int(np.argmax(unassigned))
      
        U_ij = self.get_U_ij(V_j,i)[0]
        if np.all(U_ij <= self.lb_U):
            mu_ij[i,-1] = 1
        
        else:
            U_ij = np.concatenate((U_ij, [self.lb_U]))
            U_ij_sorted_id = np.argsort(U_ij)
            j_i = U_ij_sorted_id[-1]
            w_i = U_ij[U_ij_sorted_id[-2] ]

            mu_ij[:,j_i] = 0
            mu_ij[i,j_i] = 1
            V_j[j_i] += np.maximum(self.get_V_ij(i,j_i, w_i) - V_j[j_i], self.tol)

        unassigned = mu_ij.sum(1) == 0
        n_unassigned = unassigned.sum()
        iter += 1
   
    id_i , id_j  =  np.where(mu_ij[:,:-1] == 1)
    
    U_i = np.ones(self.n) * self.lb_U
    U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j]    
   
    return mu_ij, U_i, V_j


OneToOneITU.ITU_auction = ITU_auction

In [6]:
true_mkt = OneToOneITU( n_i,m_j, (A,B), tol= 1e-14)

In [7]:
mu_obs, u_true, v_true = true_mkt.ITU_auction(0)

In [8]:
np.shape(mu_obs)

(300, 341)

In [9]:
np.all(u_true[:,None] + A * v_true[None,:] -B )>= 0

True

In [10]:
np.sum(np.maximum(B - u_true[:,None] - A * v_true[None,:] ,0))

6.333822355486518e-14

In [11]:
def conditional_moments(mu_ij, X_ijk ):
    conditional_means = np.sum(mu_ij[:,:-1,None] * X_ijk, axis = (0,1))/ mu_ij.sum()
    conditional_cov = np.sum(mu_ij[:,:-1, None, None]* (X_ijk[:,:,:,None] * X_ijk[:,:,None,:])  , axis = (0,1)) / mu_ij.sum()
    return conditional_means,conditional_cov

In [12]:
#conditional_means_obs ,conditional_covs_obs = conditional_moments(mu_obs, X_ijk )

# Estimation

In [38]:
def Q(self,θ,mu_obs,X_ijk):
    A_θ = np.exp( X_ijk[:,:,:2] @ θ[:2])
    B_θ = X_ijk[:,:,2:4] @ θ[2:4]

    simul_mkt = OneToOneITU( n_i,m_j, (A_θ,B_θ), tol= 1e-14)
    mu_θ, u_θ, v_θ = simul_mkt.ITU_auction(0)

    # mu_hat = np.argmax(mu_obs, axis = 1)
    # u_hat = np.ones(self.n) * self.lb_U

    # i_matched =  mu_hat < self.m
    # u_hat= simul_mkt.get_U_ij(v_θ, i_matched, mu_hat[i_matched])
    # #print(np.sum(np.maximum(B_θ - u_θ[:,None] - A_θ * v_θ[None,:] ,0)) )
    # CS_hat_θ = np.sum( u_θ- u_hat   )
    # print(CS_hat_θ)
    # conditional_means_θ ,conditional_cov_θ = conditional_moments(mu_θ, X_ijk )
    # moments_diff = np.sum( (self.conditional_means_obs - conditional_means_θ)**2) +  np.sum( (self.conditional_covs_obs - conditional_cov_θ)**2) 

    # Q_θ = CS_hat_θ + moments_diff


    Q_θ = ( (mu_obs[:,:-1] * (-B_θ + u_θ[:,None] + A_θ * v_θ[None,:])).sum() + 
            (mu_obs[:,-1] * u_θ ).sum() + 
            ((mu_obs[:,:-1].sum(0) == 0) * v_θ).sum() +
            np.log( mu_θ.sum()))

    print(Q_θ)
    print( θ.round(3))
    return Q_θ

OneToOneITU.Q = Q

In [39]:
true_mkt.Q(np.array([.5,.5,5,5]), mu_obs, X_ijk)

5.703782474656199
[0.5 0.5 5.  5. ]


5.703782474656199

In [40]:
def maximum_score_estimator(self,θ_0,mu_obs,X_ijk ):
    
    # self.conditional_means_obs ,self.conditional_covs_obs = conditional_moments(mu_obs, X_ijk )
    # print(self.conditional_means_obs )
    problem = sp.optimize.minimize( lambda θ: self.Q(θ,mu_obs,X_ijk)  ,θ_0)  

    return problem.x


OneToOneITU.maximum_score_estimator = maximum_score_estimator

In [41]:
true_mkt.maximum_score_estimator( np.array([.5,1,5,5]), mu_obs,X_ijk )

160.0293221226332
[0.5 1.  5.  5. ]
160.02932385713893
[0.5 1.  5.  5. ]
160.0293287666437
[0.5 1.  5.  5. ]
160.0293201319381
[0.5 1.  5.  5. ]
160.02932457325628
[0.5 1.  5.  5. ]
41.45058548802228
[0.268 0.112 5.266 4.673]
41.450585314090596
[0.268 0.112 5.266 4.673]
41.450583814406166
[0.268 0.112 5.266 4.673]
41.45058580643031
[0.268 0.112 5.266 4.673]
41.45058524316908
[0.268 0.112 5.266 4.673]
38089181035936.766
[-4.508  4.27   8.1   -2.095]
38089178889208.58
[-4.508  4.27   8.1   -2.095]
38089182730496.83
[-4.508  4.27   8.1   -2.095]
38089181109912.28
[-4.508  4.27   8.1   -2.095]
38089181051030.805
[-4.508  4.27   8.1   -2.095]
19068.06075194389
[-1.268  1.45   6.178  2.495]
19068.05972894806
[-1.268  1.45   6.178  2.495]
19068.061649942378
[-1.268  1.45   6.178  2.495]
19068.061543695258
[-1.268  1.45   6.178  2.495]
19068.05890556591
[-1.268  1.45   6.178  2.495]
218.87798321760968
[-0.225  0.541  5.558  3.974]
218.8779724710402
[-0.225  0.541  5.558  3.974]
218.87798631024

array([3.60384033e-01, 2.39092354e-01, 4.38270118e-05, 5.32614433e-05])